# GraphCypherQAChain

本笔记本演示如何使用LLMs为图形数据库提供自然语言接口，可以使用Cypher查询语言进行查询。#（注意：这里的Cypher查询语言是指一种查询语言）

您需要有一个正在运行的Neo4j实例。一种选项是在[Aura云服务中创建一个免费的Neo4j数据库实例]（https://neo4j.com/cloud/platform/aura-graph-database/）。您还可以使用[Neo4j桌面应用程序]（https://neo4j.com/download/）或运行docker容器来本地运行数据库。#（注意：这里的docker容器是指一个虚拟容器）
您可以通过执行以下脚本来运行本地docker容器：

```
docker run \
    --name neo4j \
    -p 7474:7474 -p 7687:7687 \
    -d \
    -e NEO4J_AUTH=neo4j/pleaseletmein \
    -e NEO4J_PLUGINS=\[\"apoc\"\]  \
    neo4j:latest
```

如果您使用的是docker容器，则需要等待一些时间才能启动数据库。#（注意：这里的Neo4j是一种图形数据库，而Aura和apoc是Neo4j的云服务和插件，这里均不作翻译）

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [2]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", username="neo4j", password="pleaseletmein"
)

## 向数据库添加数据

假设您的数据库为空，您可以使用Cypher查询语言来填充它。以下Cypher语句是幂等的，这意味着如果您运行它一次或多次，数据库信息将保持不变。

In [3]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

## 刷新图形模式信息
如果数据库的模式发生更改，您可以刷新生成Cypher语句所需的模式信息。

In [4]:
graph.refresh_schema()

In [5]:
print(graph.get_schema)


        Node properties are the following:
        [{'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Movie'}, {'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Actor'}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Actor)-[:ACTED_IN]->(:Movie)']
        


## 查询图形

现在我们可以使用图形Cypher QA链来询问图形了。

In [6]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [7]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}, {'a.name': 'Tom Cruise'}]

> Finished chain.


'Val Kilmer, Anthony Edwards, Meg Ryan, and Tom Cruise played in Top Gun.'

## 限制结果数量
您可以使用“top_k”参数限制来自Cypher QA链的结果数量。
默认值为10。

In [8]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, top_k=2
)

In [9]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}]

> Finished chain.


'Val Kilmer and Anthony Edwards played in Top Gun.'

## 返回中间结果
您可以使用“return_intermediate_steps”参数从Cypher QA链返回中间步骤。

In [10]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
)

In [11]:
result = chain("Who played in Top Gun?")
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}, {'a.name': 'Tom Cruise'}]

> Finished chain.
Intermediate steps: [{'query': "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})\nRETURN a.name"}, {'context': [{'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}, {'a.name': 'Tom Cruise'}]}]
Final answer: Val Kilmer, Anthony Edwards, Meg Ryan, and Tom Cruise played in Top Gun.


## 返回直接结果
您可以使用“return_direct”参数从Cypher QA链返回直接结果。

In [12]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_direct=True
)

In [13]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name

> Finished chain.


[{'a.name': 'Val Kilmer'},
 {'a.name': 'Anthony Edwards'},
 {'a.name': 'Meg Ryan'},
 {'a.name': 'Tom Cruise'}]